In [6]:
import numpy as np
import torch
from datasets import load_dataset, Value, Sequence
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
    DebertaV2Tokenizer
)
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import f1_score

# ------------------------------
# CUDA Initialization Check
# ------------------------------
if torch.cuda.is_available():
    try:
        torch.cuda.init()  # Force CUDA initialization (optional)
    except Exception as e:
        print("Error during CUDA initialization:", e)
    print("CUDA is available. Running on GPU.")
    try:
        print("GPU Name:", torch.cuda.get_device_name(0))
    except Exception as e:
        print("Could not get GPU Name:", e)
    print("GPU Count:", torch.cuda.device_count())
else:
    print("CUDA is not available. Running on CPU.")

# ------------------------------
# 1. Load the GoEmotions Dataset
# ------------------------------
dataset = load_dataset("go_emotions")
print("\nSample from training set:")
print(dataset["train"][0])  # Show a sample to inspect

# Number of labels
num_labels = dataset["train"].features["labels"].feature.num_classes
print("Number of labels:", num_labels)

# ------------------------------
# 2. Data Preprocessing
# ------------------------------
def convert_labels(label_list):
    """Convert list of label indices to a multi-hot vector of floats."""
    return [1.0 if i in label_list else 0.0 for i in range(num_labels)]

model_checkpoint = "microsoft/deberta-v3-base"
# Use a stronger checkpoint such as microsoft/deberta-v3-base
tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base")
max_length = 128  # You can experiment with longer sequences

def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    tokenized["labels"] = [convert_labels(lbls) for lbls in examples["labels"]]
    return tokenized

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.cast_column("labels", Sequence(Value("float32")))
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ------------------------------
# 3. Model Setup
# ------------------------------
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
)
model.config.problem_type = "multi_label_classification"

if torch.cuda.is_available():
    model = model.to("cuda")

# ------------------------------
# 4. Define Compute Metrics Function
# ------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Binarize predictions using threshold 0.5
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    # Compute subset accuracy (exact match)
    subset_accuracy = np.mean(np.all(predictions == labels, axis=1))
    # Compute micro and macro F1
    micro_f1 = f1_score(labels, predictions, average="micro", zero_division=0)
    macro_f1 = f1_score(labels, predictions, average="macro", zero_division=0)
    return {
        "subset_accuracy": subset_accuracy,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1
    }

# ------------------------------
# 5. Training Arguments & Trainer Setup
# ------------------------------
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # Try 5-10 epochs for better performance
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
)

def custom_data_collator(features):
    batch = DefaultDataCollator()(features)
    batch["labels"] = batch["labels"].float()
    return batch

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=custom_data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarting training...")
trainer.train()

# ------------------------------
# 6. Evaluation
# ------------------------------
print("\nEvaluating on validation set...")
val_results = trainer.evaluate()
df_val_results = pd.DataFrame([val_results])
print("Validation Results:")
print(tabulate(df_val_results, headers="keys", tablefmt="fancy_grid"))

print("\nEvaluating on test set...")
test_results = trainer.evaluate(eval_dataset=encoded_dataset["test"])
df_test_results = pd.DataFrame([test_results])
print("Test Results:")
print(tabulate(df_test_results, headers="keys", tablefmt="fancy_grid"))

# ------------------------------
# 7. Inference Example
# ------------------------------
def predict_emotions(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[0]
    probs = torch.sigmoid(logits).cpu().numpy()
    return probs

sample_text = "I am feeling really happy and excited about today!"
probs = predict_emotions(sample_text)
print("\nSample inference text:", sample_text)
print("Predicted probabilities for each emotion:")
print(probs)


CUDA is available. Running on GPU.
GPU Name: NVIDIA GeForce RTX 4070 Ti SUPER
GPU Count: 1

Sample from training set:
{'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej'}
Number of labels: 28


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Second_Sem\csc392\env_py310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saadu\.cache\huggingface\hub\models--microsoft--deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(mes


Starting training...


  1%|          | 101/13570 [00:13<27:54,  8.04it/s]

{'loss': 0.3424, 'learning_rate': 1.9852616064848935e-05, 'epoch': 0.04}


...


Evaluating on validation set...


100%|██████████| 340/340 [00:11<00:00, 29.56it/s]


Validation Results:
╒════╤═════════════╤════════════════════════╤═════════════════╤═════════════════╤════════════════╤═══════════════════════════╤═════════════════════════╤═════════╕
│    │   eval_loss │   eval_subset_accuracy │   eval_micro_f1 │   eval_macro_f1 │   eval_runtime │   eval_samples_per_second │   eval_steps_per_second │   epoch │
╞════╪═════════════╪════════════════════════╪═════════════════╪═════════════════╪════════════════╪═══════════════════════════╪═════════════════════════╪═════════╡
│  0 │   0.0844298 │               0.460192 │         0.57608 │        0.441173 │        11.5908 │                   468.128 │                  29.333 │       5 │
╘════╧═════════════╧════════════════════════╧═════════════════╧═════════════════╧════════════════╧═══════════════════════════╧═════════════════════════╧═════════╛

Evaluating on test set...


100%|██████████| 340/340 [00:11<00:00, 29.77it/s]

Test Results:
╒════╤═════════════╤════════════════════════╤═════════════════╤═════════════════╤════════════════╤═══════════════════════════╤═════════════════════════╤═════════╕
│    │   eval_loss │   eval_subset_accuracy │   eval_micro_f1 │   eval_macro_f1 │   eval_runtime │   eval_samples_per_second │   eval_steps_per_second │   epoch │
╞════╪═════════════╪════════════════════════╪═════════════════╪═════════════════╪════════════════╪═══════════════════════════╪═════════════════════════╪═════════╡
│  0 │   0.0840901 │                0.46066 │        0.576743 │        0.441563 │        11.4569 │                   473.688 │                  29.676 │       5 │
╘════╧═════════════╧════════════════════════╧═════════════════╧═════════════════╧════════════════╧═══════════════════════════╧═════════════════════════╧═════════╛

Sample inference text: I am feeling really happy and excited about today!
Predicted probabilities for each emotion:
[5.21914177e-02 1.28694540e-02 8.52493919e-04 1.108821

In [5]:
# Use a specific prompt for inference
prompt_text = "I want a burger"
probs = predict_emotions(prompt_text)

# Get the label names from the dataset (GoEmotions)
label_names = dataset["train"].features["labels"].feature.names

threshold = 0.5 

# Convert probabilities to binary predictions
predicted_binary = (probs > threshold).astype(int)

# For multi-label classification, there may be multiple labels predicted.
predicted_labels = [label_names[i] for i, pred in enumerate(predicted_binary) if pred == 1]

print("\nPredicted labels (threshold={}):".format(threshold))
print(predicted_labels)



Predicted labels (threshold=0.5):
['desire']


In [3]:
import transformers
print(transformers.__version__)

4.31.0
